In [23]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [24]:
# Get the data for a page.
def getPageData(url):
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    css_selector = '.awrapper .listItemContainer .listItemLink'
    data = []
    i = 0
    for item in soup.select(css_selector):
        i += 1
        title = item.select_one('span.title').text.strip()
        price = item.select_one('span.price').text.strip()
        price = re.sub('[^0-9]', '', price)
        image = item.select_one('img.cover')['src']
        if title and price:
            data.append([title, price, 'https:' + image])
    return data

In [25]:
base_url = 'https://bazar.bg/obiavi/gradski-velosipedi/varna?condition=2'
count_pages = 5
data = []
for i in range(count_pages):
    cur_page = i + 1
    print(f'Get page {cur_page} of {count_pages}')
    if cur_page == 1:
        url = base_url
    else:
        url = base_url + '&page=' + str(cur_page)
    data += getPageData(url)

Get page 1 of 5
Get page 2 of 5
Get page 3 of 5
Get page 4 of 5
Get page 5 of 5


In [26]:
df = pd.DataFrame(data, columns=['title', 'price', 'image'])
df.sort_values(by='price')
df.to_excel('bikes.xlsx')
df.head()

,title,price,image
0,Avenue spirit 1000,1300,https://cdn1.focus.bg/bazar/10/fp/106de41fa4f6...
1,велосипед 26 цола CONWAY-една година гаранция,155,https://cdn1.focus.bg/bazar/d2/fp/d2c10a8fca2b...
2,Велосипед 26 цола,250,https://cdn1.focus.bg/bazar/61/fp/61c5ca62cd6a...
3,велосипед Cannondale USA,480,https://cdn1.focus.bg/bazar/33/fp/338062f566da...
4,Велосипед Ultra phantom 26 цола,170,https://cdn1.focus.bg/bazar/97/fp/9714505365f2...
